In [ ]:
import sys
sys.path.append('/home/fjargsto/AcousticBEM/Python')

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

from InteriorHelmholtzSolverRAD_C import *

class ImpedanceTube(object):
    def __init__(self, length = 1.0, radius = 0.1, elements_l = 100, elements_r = 10,
                impedance = 0.0 + 0.0j, velocityAmplitude = 1.0):
        nElements = elements_l + 2 * elements_r
        # key simulation parameters
        self.density   = 1.205 # density of air [kg/m^3]
        self.velocity  = 344.0 # speed of sound in air [m/s]
        self.frequency = 1000  # 1000 Hz 
        
        self.impedance = impedance
        self.velocityAmplitude = velocityAmplitude

        # key boundary parameters
        self.length = length
        self.radius = radius
        self.elements_l = elements_l
        self.elements_r = elements_r

        # implementation specific parameters
        self.bUseOptimized = True

        # creation of boundary discretization
        self.aVertex, self.aEdge = self.Cylinder(length, radius, elements_l, elements_r)
        self.boundaryCondition = BoundaryCondition(nElements)
        self.boundaryCondition.alpha.fill(0.0)
        self.boundaryCondition.beta.fill(1.0)
        self.boundaryCondition.f.fill(0.0)
        self.boundaryCondition.f[-elements_r:] = self.velocityAmplitude
        self.boundaryCondition.f[:elements_r] = 0.0
        self.boundaryCondition.alpha[:elements_r] = 1.0
        self.boundaryCondition.beta[:elements_r] = self.impedance
        
        # definition of incident fields on boundary
        self.boundaryIncidence = BoundaryIncidence(nElements)
        self.boundaryIncidence.phi.fill(0.0) # no incoming velocity potential on boundary                                              
        self.boundaryIncidence.v.fill(0.0)   # no incoming velocity on boundary                                                        

        # description of the interior field solution being sought
        delta = length / elements_l
        self.aInteriorPoints = np.zeros((2 * elements_l - 1, 2), dtype=np.float32)
        self.aInteriorPoints[:, 1] = np.linspace(delta/2, length - delta/2, 2 * elements_l - 1, True)
        self.aInteriorIncidentPhi = np.zeros(self.aInteriorPoints.shape[0])
        
    # Solve problem as described by object state
    def solve(self):
        if self.bUseOptimized:
            self.tubeProblem = InteriorHelmholtzSolverRAD_C(self.aVertex, self.aEdge, 
                                                          self.velocity, self.density)
        else:
            self.tubeProblem = InteriorHelmholtzSolverRAD(self.aVertex, self.aEdge, 
                                                          self.velocity, self.density)
        k = self.tubeProblem.frequencyToWavenumber(self.frequency)
        print "starting solve boundary"
        self.boundarySolution = self.tubeProblem.solveBoundary(k, self.boundaryCondition,
                                                               self.boundaryIncidence)
        print "completed boundary solution, starting solve interior"
        self.interiorPhi = self.tubeProblem.solveInterior(self.boundarySolution, 
                                                          self.aInteriorIncidentPhi, 
                                                          self.aInteriorPoints)
        print "completed interior solution"

    def showPressureMap(self, t = 0.0):
        fig = plt.figure()
        plt.plot(self.aInteriorPoints[:,1], 
                 self.tubeProblem.soundPressure(self.boundarySolution.k, self.interiorPhi, t).real)
        plt.show()  

    @classmethod
    def Cylinder(cls, length, radius, elements_l, elements_r):
        nlements = elements_l + 2 * elements_r
        # boundary elements are edges
        aEdge = np.empty((nlements, 2), dtype=np.int32)
        aEdge[:, 0] = range(nlements)
        aEdge[:, 1] = range(1, nlements+1)
        
        aVertex = np.empty((nlements + 1, 2), dtype=np.float32)
        # up the r-Axis
        aVertex[0:elements_r, 0] = np.linspace(0.0, radius, elements_r, False)
        aVertex[0:elements_r, 1] = length
        # over to the right along top edge
        aVertex[elements_r:elements_r+elements_l, 0] = radius
        aVertex[elements_r:elements_r+elements_l, 1] = np.linspace(length, 0.0, elements_l, False)
        # back down to x-axis
        aVertex[elements_r+elements_l:nlements, 0] = np.linspace(radius, 0.0, elements_r, False)
        aVertex[elements_r+elements_l:nlements, 1] = 0.0
        
        return aVertex, aEdge


In [ ]:
tube = ImpedanceTube(1.0, 0.1, impedance = 10-5j)
tube.solve()
tube.showPressureMap()

In [ ]:
def p(x):
    q = tube.density * tube.velocity 
    k = tube.tubeProblem.frequencyToWavenumber(tube.frequency)
    return q * tube.velocityAmplitude * (tube.impedance * np.cos(k * (tube.length - x)) \
        - 1j * q * np.sin(k * (tube.length - x))) / (q * np.cos(k * tube.length) - 1j * \
                                                    tube.impedance * np.sin(k * tube.length))
xVals = np.linspace(0.0, tube.length, tube.elements_l)
yVals = p(xVals).real
fig = plt.figure()
plt.plot(xVals, yVals)
plt.show()
            